# Cálculo de coeficientes gumbel en base a los valores extremos de las respuestas dinámicas $F_X$ y $M_Y$.

## Importando librerías

In [1]:
from scipy.stats import gumbel_r, kstest
import pandas as pd
import numpy as np
import math

## 1. Importando datos

In [2]:
fx = pd.read_excel(r'..\data\Stats_responses_FX_MY.xlsx', sheet_name = 'Fx', usecols = 'A,D')
my = pd.read_excel(r'..\data\Stats_responses_FX_MY.xlsx', sheet_name = 'My', usecols = 'A,D')

In [3]:
#pasando de kN a MN
fx['Max'] = fx['Max'] / 1000
my['Max'] = my['Max'] / 1000

## 2. Creando funciones para estimar los parametros de ajuste de Gumbel y prueba de hipotesis

In [4]:
def get_gumbcoef(maxLst):
    params =gumbel_r.fit(maxLst)
    D, pval = kstest(maxLst, 'gumbel_r', args = (params))
    
    return [params[0], params[1], pval]

In [5]:
uw_labels = fx['Uw'].unique() #Tambien puede ser my['Uw'].unique()

### 2.1. Obteniendo coeficientes de ajuste para $F_X$

In [6]:
fit_data_fx =[]

for u in uw_labels:
    tmp_max = fx.loc[fx['Uw'] == u]
    result = get_gumbcoef(tmp_max['Max'])
    
    fit_data_fx.append([u, result[0], result[1], result[2]])

In [7]:
fitfx_df = pd.DataFrame(fit_data_fx , columns = ['Uw', 'muG', 'betaG', 'pvalue'])

In [8]:
fitfx_df.round(3)

,Uw,muG,betaG,pvalue
0,7.0,0.529,0.038,0.665
1,10.0,0.865,0.019,0.738
2,11.4,0.905,0.019,0.375
3,12.0,0.917,0.021,0.792
4,15.0,0.906,0.040,0.129
5,18.0,0.808,0.070,0.367
6,22.0,0.754,0.039,0.709
7,25.0,0.784,0.036,0.948


### 2.2. Obteniendo coeficientes de ajuste para $M_Y$

In [9]:
fit_data_my =[]

for u in uw_labels:
    tmp_max = my.loc[my['Uw'] == u]
    result = get_gumbcoef(tmp_max['Max'])
    
    fit_data_my.append([u, result[0], result[1], result[2]])

In [10]:
fitmy_df = pd.DataFrame(fit_data_my , columns = ['Uw', 'muG', 'betaG', 'pvalue'])

In [11]:
fitmy_df.round(3)

,Uw,muG,betaG,pvalue
0,7.0,43.848,3.427,0.914
1,10.0,72.934,1.619,0.495
2,11.4,76.281,1.404,0.360
3,12.0,77.434,1.559,0.520
4,15.0,76.911,3.150,0.174
5,18.0,65.970,6.748,0.526
6,22.0,59.274,3.555,0.609
7,25.0,59.824,3.738,0.087


In [12]:
fitfx_df.to_excel('(Resultados) Datos de ajuste Gumbel - FX.xlsx', index = False)
fitmy_df.to_excel('(Resultados) Datos de ajuste Gumbel - MY.xlsx', index = False)

## 3. Hallando el periodo de retorno del viento para aplicarlo en la extrapolación

In [13]:
def get_returnperiod(Uw, wsc, exp_wei, k_wei):
    #exp_wei: parametro de forma; k_wei: parametro de escala
    
    uten = Uw*(10/90)**wsc                       #Se considera que el hub esta a 90 m    
    F = 1-math.exp(-(uten/k_wei)**exp_wei)    
    N = 10/((1-F)*365.25*24*60)
    
    return N

In [14]:
#parametros para el periodo de retorno
wsc = 0.10
exp_wei = 2
k_wei =7

## 4. Aplicando extrapolación a 50 años

### FX

In [15]:
fx_sum = fitfx_df[['Uw','muG']].copy()

In [16]:
# Calculando el periodo de retorno
fx_sum['N'] = fx_sum['Uw'].apply(lambda x: get_returnperiod(x, wsc, exp_wei, k_wei))

In [17]:
# Calculano el valor de respuesta dinamica extrapolado
fx_sum['FX (LT)'] = fx_sum['muG'] + fitfx_df['betaG'] * np.log(6 * 50 / fx_sum['N'])

In [18]:
fx_sum

,Uw,muG,N,FX (LT)
0,7.0,0.528927,0.000036,1.136027
1,10.0,0.865304,0.000071,1.152238
2,11.4,0.904766,0.000105,1.188236
3,12.0,0.917478,0.000126,1.221990
4,15.0,0.906275,0.000367,1.451310
5,18.0,0.807844,0.001347,1.673296
6,22.0,0.753583,0.011050,1.149827
7,25.0,0.784337,0.070575,1.086596


### MY

In [19]:
my_sum = fitmy_df[['Uw','muG']].copy()

In [20]:
# Calculando el periodo de retorno
my_sum['N'] = my_sum['Uw'].apply(lambda x: get_returnperiod(x, wsc, exp_wei, k_wei))

In [21]:
# Calculano el valor de respuesta dinamica extrapolado
my_sum['FX (LT)'] = my_sum['muG'] + fitmy_df['betaG'] * np.log(6 * 50 / my_sum['N'])

In [22]:
my_sum

,Uw,muG,N,FX (LT)
0,7.0,43.847587,0.000036,98.432150
1,10.0,72.934499,0.000071,97.632690
2,11.4,76.281084,0.000105,97.152836
3,12.0,77.434042,0.000126,100.315520
4,15.0,76.910515,0.000367,119.794252
5,18.0,65.969592,0.001347,149.065315
6,22.0,59.273912,0.011050,95.564408
7,25.0,59.823530,0.070575,91.052611
